In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("us.env"))  # read local .env file

In [2]:
from pathlib import Path
from tqdm import tqdm
import sys

sys.path.append(Path("..").resolve().as_posix())

## Prepare LLMs

In [3]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings
from langchain_openai import AzureChatOpenAI

openai_api_version = "2023-07-01-preview"

# gpt35 = AzureOpenAI(
#     azure_deployment="gpt-35-turbo-1106-edgedb-1",
#     openai_api_version=openai_api_version,
# )

gpt35 = AzureOpenAI(
    azure_deployment="gpt-35-turbo-1106",
    openai_api_version=openai_api_version,
)

gpt4 = AzureOpenAI(
    azure_deployment="gpt-4-1106",
    openai_api_version=openai_api_version,
)

langchain_gpt35 = AzureChatOpenAI(
    temperature=0.1,
    azure_deployment="gpt-35-turbo-1106",
    openai_api_version=openai_api_version,
)

langchain_gpt4 = AzureChatOpenAI(
    temperature=0.1,
    azure_deployment="gpt-4-1106",
    openai_api_version=openai_api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    deployment_name="text-embedding-ada-002",
    api_version=openai_api_version,
    embed_batch_size=100,
)

Settings.llm = gpt35
Settings.embed_model = embed_model

## Assemble the RAG

In [4]:
from src.core.index import build_index
from src.core.retriever import build_retriever
from src.core.postprocessor import build_context_filter
from src.core.query_engine import build_query_engine

In [5]:
lib_path = Path("../../docs_md").resolve()
assert lib_path.exists()

persist_path = Path("index_storage").resolve()
collection_name = "dev1"

index, full_nodes_dict = build_index(
    persist_path=persist_path, collection_name=collection_name, lib_path=lib_path
)

retriever = build_retriever(
    index=index, full_nodes_dict=full_nodes_dict, llm=gpt35, verbose=False, top_k=10
)

context_filter = build_context_filter(
    langchain_light=langchain_gpt35, langchain_heavy=langchain_gpt4
)

query_engine = build_query_engine(
    retriever=retriever,
    llm=gpt4,
    postprocessors=[context_filter],
    langchain_light=langchain_gpt35,
    langchain_heavy=langchain_gpt4,
)

In [6]:
response = query_engine.query("Tell me about a basic select")
print(response.response)

Using query str: Tell me about a basic select
Using filters: [('section', '==', 'edgeql_and_sdl')]


## Run benchmarks

In [ ]:
easy_query_path = Path("benchmarks/tutorial_benchmark.txt").resolve()
# hard_query_path = Path("benchmarks/user_benchmark.txt").resolve()
hard_query_path = Path("benchmarks/new_hard.txt").resolve()

In [ ]:
with easy_query_path.open("r") as f:
    easy_queries = [query.strip() for query in f.readlines()]

with hard_query_path.open("r") as f:
    hard_queries = f.read().split(">>>SEPARATOR<<<")